# Sample 데이터 만들기
- pandas 설치 후 import 하고 사용하려고 하면 kernel이 crashed 났다면서 에러 발생
- 다른 곳에서 코드 실행 후 데이터만 가져옴 

In [2]:
# import numpy as np
# import pandas as pd

# # Define the number of users and unique items
# num_users = 1000
# num_items = 20

# # Generate random user IDs and item IDs
# user_ids = np.arange(1, num_users + 1)
# item_ids = np.arange(1, num_items + 1)

# # Create random interaction data
# data = {
#     'user_id': np.random.choice(user_ids, size=num_users * 10),
#     'item_id': np.random.choice(item_ids, size=num_users * 10),
# }

# # Create a pandas DataFrame from the data
# df = pd.DataFrame(data).drop_duplicates()

# # Display the first few rows of the generated data
# print(df.head())


# #grouping all interactions by a user as list
# df = df.groupby(['user_id'])['item_id'].agg(list).reset_index()

# #creating OHE 
# df['item_id'] = df['item_id'].transform(lambda x: [0 if y+1 not in x else y+1 for y in range(20)])


# #save csv
# df.to_csv('dummy_data.csv',index=False)


In [19]:
import os
from dotenv import load_dotenv, find_dotenv

# import chromadb
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
import chromadb
# 그냥 pip(3) install chromadb 하면 전역에 깔리는데..
# python3 -m pip install chromadb 하니까 됨

In [6]:
#data loader
loader = CSVLoader(file_path="dummy_data.csv")
data = loader.load()


In [7]:
# document 형식으로 data load
data

[Document(metadata={'source': 'dummy_data.csv', 'row': 0}, page_content='user_id: 1\nitem_id: [1, 0, 0, 0, 0, 6, 0, 8, 0, 0, 0, 12, 0, 14, 0, 16, 17, 18, 0, 0]'),
 Document(metadata={'source': 'dummy_data.csv', 'row': 1}, page_content='user_id: 2\nitem_id: [0, 2, 3, 0, 5, 6, 0, 0, 0, 10, 11, 0, 0, 0, 15, 0, 0, 0, 0, 20]'),
 Document(metadata={'source': 'dummy_data.csv', 'row': 2}, page_content='user_id: 3\nitem_id: [0, 2, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 14, 15, 0, 0, 18, 0, 0]'),
 Document(metadata={'source': 'dummy_data.csv', 'row': 3}, page_content='user_id: 4\nitem_id: [0, 2, 3, 4, 0, 6, 0, 0, 0, 0, 0, 0, 13, 14, 0, 0, 17, 0, 0, 20]'),
 Document(metadata={'source': 'dummy_data.csv', 'row': 4}, page_content='user_id: 5\nitem_id: [1, 0, 0, 4, 0, 0, 0, 0, 9, 10, 11, 0, 0, 14, 15, 0, 17, 18, 0, 20]'),
 Document(metadata={'source': 'dummy_data.csv', 'row': 5}, page_content='user_id: 6\nitem_id: [1, 0, 0, 0, 0, 0, 0, 0, 9, 10, 11, 0, 13, 0, 0, 0, 17, 18, 19, 0]'),
 Document(metadata={'so

In [8]:
#data transformers
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [9]:
load_dotenv()

True

In [10]:
find_dotenv()

'c:\\Users\\thdus\\work\\langchain-academy\\.env'

In [20]:

#embeddings model, this can be aa local LLM as well
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

#Vector DB
docsearch = Chroma.from_documents(texts, embeddings)


In [13]:
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/28/8e/5c186c77bf749b6fe0528385e507e463f1667543328d76fd00a49e1a4e6a/chromadb-0.6.3-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/84/c2/80633736cd183ee4a62107413def345f7e6e3c01563dbca1417363cf957e/build-1.2.2.post1-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.6 from https://files.pythonhosted.org/packages/3e/79/1bce519cf186112d6d5ce2985392a89528c6e1e9332d680bf752694a4cdf/chroma_hnswlib-0.7.6-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/52/b3/7e4df40e585df024fac2f80d1a2d579c854ac37109675db2b0cc22c0bb9e/fastapi-0.115.6-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/61/14/33a3a1352cfa71812a3a21e8c9bfb83f60b00


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\thdus\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [23]:

#Retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
78,"[0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 12, 0, 14, 0, 0, 17, 0, 0, 20]"
79,"[1, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 12, 13, 0, 15, 0, 0, 18, 19, 0]"

In [24]:
qa.run('Suggest 2 articles to user-id 78 using given data which it has not seen.\
Follow this approach 1: Find similar Users and 2: sugest new articles from similar users.\
Also give a reason for suggestion').split('.')

C:\Users\LGCNS\AppData\Local\Temp\ipykernel_20748\1554469534.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run('Suggest 2 articles to user-id 78 using given data which it has not seen.\


['\n\n1',
 ' User 178 has seen articles 2, 3, 4, 7, 9, 10, 14, 17, and 18 which user 78 has not seen',
 ' These articles could be suggested as they are popular among user 178 and there is a high chance that user 78 may also like them',
 '\n\n2',
 ' User 79 has seen articles 1, 2, 4, 12, 13, 15, and 18 which user 78 has not seen',
 ' These articles could be suggested as they have similar interests and preferences, and there is a possibility that user 78 may also find them interesting',
 ' ']